In [ ]:
def down_block(x, filters, kernel_size=(3, 3), padding="same", strides=1):
    c = keras.layers.Conv2D(filters, kernel_size, padding=padding, strides=strides, activation="relu")(x)                   
    c = keras.layers.Conv2D(filters, kernel_size, padding=padding, strides=strides, activation="relu")(c)

    p = keras.layers.MaxPool2D((2, 2), (2, 2))(c)
    return c, p

def up_block(x, skip, filters, kernel_size=(3, 3), padding="same", strides=1):
    us = keras.layers.UpSampling2D((2, 2))(x)
    concat = keras.layers.Concatenate()([us, skip])
    c = keras.layers.Conv2D(filters, kernel_size, padding=padding, strides=strides, activation="relu")(concat)
    c = keras.layers.Conv2D(filters, kernel_size, padding=padding, strides=strides, activation="relu")(c)


def bottleneck(x, filters, kernel_size=(3, 3), padding="same", strides=1):
    c = keras.layers.Conv2D(filters, kernel_size, padding=padding, strides=strides, activation="relu")(x)  
    c = BatchNormalization(epsilon=1e-3,beta_initializer=Constant(0.0),gamma_initializer=Constant(1.0),momentum=0.5)(c)
    c1= keras.layers.Conv2D(filters,(3,3),strides=1,padding="same",dilation_rate=2)(x)
    c5= keras.layers.Conv2D(filters,(3,3),strides=1,padding="same",dilation_rate=4)(x)
    c2= keras.layers.Conv2D(filters,(3,3),strides=1,padding="same",dilation_rate=16)(x)
    c3= keras.layers.Conv2D(filters,(3,3),strides=1,padding="same",dilation_rate=8)(x)
    c4= keras.layers.Conv2D(filters,(3,3),strides=1,padding="same",dilation_rate=32)(x)
    c= Concatenate()([c,c1,c2,c3,c4,c5])  
    return c

In [ ]:
def ASPPUNet():
    f = [32,64,128,256]
    #inputs = keras.layers.Input((image_size, image_size, 3))
    inputs= tf.keras.layers.Input((512,512,3)) 
    
    p0 = inputs
    c1, p1 = down_block(p0, f[0]) #128 -> 64
    c2, p2 = down_block(p1, f[1]) #64 -> 32
    c3, p3 = down_block(p2, f[2]) #32 -> 16
    #c4, p4 = down_block(p3, f[3]) #16->8
    
    #bn = bottleneck(p4, f[4])
    bn = bottleneck(p3, f[3])
    
    #u1 = up_block(bn, c4, f[3]) #8 -> 16
    u2 = up_block(bn, c3, f[2]) #16 -> 32
    u3 = up_block(u2, c2, f[1]) #32 -> 64
    u4 = up_block(u3, c1, f[0]) #64 -> 128
    
    outputs = Conv2D(1, (1, 1), padding="same", activation="sigmoid")(u4)
    model = keras.models.Model(inputs, outputs)
    return model

In [ ]:
model = ASPPUNet()
adam = optimizers.Adam(lr=0.0001 , decay=5*1e-7)
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["acc"])
model.summary()